# Lab 1B – Encriptado y Decriptado de Texto

In [25]:
alphabet = "abcdefghijklmnñopqrstuvwxyz"

def cleanText(text):
    text = text.replace("á", "a").replace("é", "e").replace("í", "i").replace("ó", "o").replace("ú", "u").lower()

    return ''.join(char for char in text if char.isalpha())

def frequencyAnalysis(cipher):
    frequency = {}
    for char in alphabet:
        frequency[char] = cipher.count(char) / len(cipher) * 100
    return frequency

theoreticalFreq = {}
with open('sp_frequencies.txt', 'r', encoding='utf-8') as file:
    next(file)
    for linea in file:
        partes = linea.strip().split('\t')
        if len(partes) == 2:
            letra, frecuencia = partes
            theoreticalFreq[letra.lower()] = float(frecuencia.strip('%'))

def metric(text):
    realFreq = frequencyAnalysis(text)
    error = sum(abs(realFreq.get(letra, 0) - theoreticalFreq.get(letra, 0)) for letra in theoreticalFreq)
    return error

# Fuerza bruta Ceasar

In [47]:
def caesarDecrypt(cipher, key):
    return ''.join(alphabet[(alphabet.index(char) - key) % len(alphabet)] for char in cipher)

def brutteForceCeasar(cipher):
    cipher = cleanText(cipher)
    results = []
    for i in range(len(alphabet)):
        message = caesarDecrypt(cipher, i)
        results.append((i, metric(message), message))
    results.sort(key=lambda x: x[1])
    for key, error, message in results[:5]:
        print(f"Key ({key}) (Error={error:.2f}): {message}")
        """ print('Key (' + str(i) + '): ' + caesarDecrypt(cipher, i))
    if (input("¿Es este el mensaje original? (s/n): ") == "s"):
        return
    else:
        for i in range(0, alphabet.index(start)):
            print(caesarDecrypt(cipher, i)) """
        

# jjqaopñlhwxañejplzecepwhajyljopwjpaarlhqyeljhwwcehezwzyñemplcñwbeywyñemplwcehezwzmwñwwxñarewñaoqjiaywjeoilzazabajowyñqyewhloxñejzwhwywmwyezwzzailzebeywññwmezwiajpaahqolzawhclñepilouyhwraoyñemplcñwbeyloqjwwyyeljjayaowñewmwñwwjpeyemwñjlowhwobqpqñwowiajwvwozayexañoacqñezwz
brutteForceCeasar(input("Ingrese el mensaje cifrado: "))

Key (23) (Error=33.16): nnuestrolaberintodigitalenconstanteevolucionlaagilidadcriptograficacriptoagilidadparaabreviaresunmecanismodedefensacrucialosbrindalacapacidaddemodificarrapidamenteelusodealgoritmosyclavescriptograficosunaaccionnecesariaparaanticiparnosalasfuturasamenazasdeciberseguridad
Key (12) (Error=93.69): xxfodeczvlmocsxezñsqselvoxnzxdelxeoogzvfnszxvllqsvsñlñncsaezqclpsnlncsaezlqsvsñlñalcllmcogslcodfxwonlxsdwzñoñopoxdlncfnslvzdmcsxñlvlnlalnsñlññowzñspsnlcclasñlwoxeoovfdzñolvqzcsewzdjnvlgodncsaezqclpsnzdfxllnnszxxonodlcslalcllxesnsalcxzdlvldpfefcldlwoxlkldñonsmocdoqfcsñlñ
Key (7) (Error=98.03): ccktijheapqthxcjesxvxjpatcrecijpcjttleakrxecappvxaxspsrhxfjevhpuxrprhxfjepvxaxspsfphppqhtlxphtikcbtrpcxibeststutciprhkrxpaeiqhxcspaprpfprxspsstbesxuxrphhpfxspbtcjttakiestpavehxjbeiñrapltirhxfjevhpuxreikcpprrxecctrtiphxpfphppcjxrxfphceipapiukjkhpipbtcpopistrxqthitvkhxsps
Key (8) (Error=98.49): bbjshigdzopsgwbidrwuwiozsbqdbhiobisskdzjqwdbzoouwzwrorqgweidugotwqoqgweidouwzwroreogoopgskwog

# Fuerza bruta Afínes

In [48]:
from sympy import mod_inverse

def affineDecrypt(cipher, a, b):
    aInverse = mod_inverse(a, len(alphabet))
    return ''.join(alphabet[(aInverse * (alphabet.index(char) - b)) % len(alphabet)] for char in cipher)

def brutteForceAffine(cipher):
    cipher = cleanText(cipher)
    frequency = frequencyAnalysis(cipher)
    frequency = dict(sorted(frequency.items(), key=lambda item: item[1], reverse=True))
    frequency = list(frequency.keys())
    results = []
    X_f = alphabet.index('e')
    C_f = alphabet.index(frequency[0])
    X_s = alphabet.index("a")
    C_s = alphabet.index(frequency[1])

    for a in [1,2,4,5,7,8,10,11,13,14,16]:
        diff_X = (X_f - X_s) % len(alphabet)
        diff_C = (C_f - C_s) % len(alphabet)
        
        if diff_X == 0:
            continue

        if (diff_C * mod_inverse(diff_X, len(alphabet))) % len(alphabet) != a:
            continue
        
        for b in range(1, 17):
            message = affineDecrypt(cipher, a, b)
            results.append((a, b, metric(message), message))
            """ if "extraerdelosataques" in message:
                return f'Key({a}, {b}): {message}' """
    
    results.sort(key=lambda x: x[2])
    for a, b, error, message in results[:5]:
        print(f"Key ({a}, {b}) (Error={error:.2f}): {message}")

#print(brutteForceAffine(input("Ingrese el mensaje cifrado: ")))
# zigiuñpjdipoziyicbdoddiyxbñhjorbpbdodciñmidiiahxoixdipjcohosmicioxhtpiidmrdipixxjxxiqipjmzoditbpbdodyxmybopsmiñixubhbjopjcohoyozhicpiixpouiujxbodiubpicdicbchiuocfxjtoxbznjxuoybjzqopbjcoociuñxicocsmipjrxoxjzxiymñixoxcixoñbdouizhinmixjzpocsmidiujchxoxjzyxbñhjorbpbdodxiiuñpokozdjxoñbdouizhicmcypoqicyxbñhjrxonbyocfoprjxbhujcyjuñxjuihbdjcñjxjhxjczmiqjcfcirmxjcchibzybdizhicbxqiyjujmzypoxjigiuñpjdipobuñjxhozybodipoyxbñhjorbpbdodizzmichxotohoppoyjzhbzmoyjzhxopocouizokocdbzoubyocdiybtixcirmxbdod
brutteForceAffine(input("Ingrese el mensaje cifrado: "))

Key (5, 15) (Error=19.08): nejemplodelanecesidaddecriptoagilidadsepuedeextraerdelosataqueseartbleedugdelerrorrevelounadebilidadcrucialquepermitioalosatacantesleerlamemoriademilesdesistemasyrobarinformacionvaliosaasempresasquelograronrecuperarserapidamentefueronlasquedemostraroncriptoagilidadreemplazandorapidamentesusclavescriptograficasyalgoritmoscomprometidosporotrosnuevosysegurossteincidentesirvecomounclaroejemplodelaimportanciadelacriptoagilidadennuestrabatallacontinuacontralasamenazasdinamicasdeciberseguridad
Key (5, 16) (Error=96.88): ctytbfaestapctrtixspsstrhxfjepvxaxspsitfktsttnjhpthstaeipjpgktitphjqattskvstathhehhtltaekcpstqxaxspsrhkrxpagktfthbxjxepaeipjprpcjtiatthapbtbehxpstbxatistixijtbpiñheqphxcuehbprxeclpaxeippitbfhtipigktaevhphechtrkfthphithpfxspbtcjtukthecapigktstbeijhphecrhxfjepvxaxspshttbfapopcsehpfxspbtcjtikirapltirhxfjevhpuxrpiñpavehxjbeirebfhebtjxseifehejheicktleiñitvkheiijtxcrxstcjtixhltrebekcraphetytbfaestapxbfehjpcrxpstaprhxfjepvxaxspstccktijhpqpjpaaprecjxckprecjh

# Fuerza bruta Vigeneré

In [41]:
x = "pabilo, pábilo, pablar, pablos, pábulo, pacado, pacana, pacata, pacato, pacaya, pacéis, paceño, pacerá, paceré, pachón, pacían, pacías, pacida, pacido, pación, pacora, pacota, pactad, pactan, pactar, pactas, pacten, pactes, pactos, padece, padecí, padree, padres, padrón, paella, pafias, pafios, paflón, pagaba, pagada, pagado, pagáis, pagana, pagano, pagara, pagará, pagare, pagaré, pagase, pagaya, pagaza, página, pagine, paginé, pagino, paginó, pagoda, pagote, paguen, pagues, paguro, paicos, pailón, painel, paipai, paipay, pairar, pairos, países, pajado, pájara, pájaro, pajaza, pajazo, pajead, pajean, pajear, pajeas, pajeen, pajees, pajera, pajero, pajiza, pajizo, pajosa, pajoso, pajote, pajuil, pajuno, pajuzo, palada, palado, palazo, palcos, palead, paleal, palean, palear, paleas, paleen, palees, palera, palero, paleta, paleto, paletó, paliad, palian, paliar, palias, pálida, pálido, palier, palios, paliza, pallar, pallas, pallón, palmad, palman, palmar, palmas, palmea, palmee, palmeé, palmen, palmeo, palmeó, palmos, palomo, palote, palpad, palpan, palpar, palpas, palpen, palpes, palpos, palqui, palude, paludo, pambil, pamela, pamema, pampas, pampón, pánace, panada, panado, pancho, pancos, pandad, pandan, pandar, pandas, pandee, panden, pandeo, panela, panera, panero, pangal, pangue, pánico, panizo, panoja, panoli, panoso, panque, panqué, pansas, panudo, panzas, panzón, pañera, pañero, pañete, pañito, pañosa, pañoso, papaba, papada, papado, papáis, papara, papará, papare, paparé, páparo, papase, papaya, papayo, papear, papéis, papera, papero, papila, papión, papiro, pápiro, papisa, papuda, papudo, pápula, paquee, paqueo, paraba, paraca, parada, parado, paráis, paraje, páramo, parara, parará, parare, pararé, parase, parata, parcas, parcha, parche, parché, parcho, parchó, parcir, parcos, pardal, pardas, pardea, pardee, pardeé, pardeo, pardeó, pardos, paread, parean, parear, pareas, parece, parecí, pareen, parees, paréis, pareja, parejo, pareos, paresa, párese, pargos, parián, parían, parias, parías, parida, parido, párido, paripé, parirá, pariré, parlad, parlan, parlar, parlas, parlen, parles, parlón, parnés, parola, pároli, parpar, parque, parqué, parqui, parrad, parral, parran, parrar, parras, parrel, parren, parrón, parsec, pársec, parser, partan, partas, partee, parten, partes, partía, partid, partil, partió, partir, partís, partos, parvas, parvos, pasaba, pasada, pasado, pasáis, pasaje, pasara, pasará, pasare, pasaré, pasase, pascal, pascua, pasead, pasean, pasear, paseas, paseen, pasees, paséis, páseme, paseos, pasera, pasero, pásese, pasión, pasito, pasiva, pasivo, pasmad, pasman, pasmar, pasmas, pasmen, pasmes, pasmón, pasmos, pasoso, pasota, pasote, paspié, pássim, pastad, pastan, pastar, pastas, pastee, pastel, pasten, pastes, pastón, pastor, pastos, pasudo, pataca, pataco, patada, pataje, patajú, pataos, patata, patead, patean, patear, pateas, pateen, patees, patena, pateos, patera, pátera, patero, pateta, patina, pátina, patine, patiné, patino, patinó, patios, patita, patito, patojo, patona, patosa, patoso, patota, patria, patrio, patrón, patuco, patuda, patudo, paujil, paular, pausad, pausan, pausar, pausas, pausen, pauses, pautad, pautan, pautar, pautas, pauten, pautes, pavada, pavana, pavera, pavero, pavesa, pavías, pávida, pávido, pavona, pavone, pavoné, pavono, pavonó, pavura, payada, payana, payasa, payaso, payesa, pazcón, pazote"
words = []
for y in x.split(','):
    words.append(cleanText(y))

In [46]:
def vigenereDecrypt(cipher, key):
    return ''.join(alphabet[(alphabet.index(char) - alphabet.index(key[i % len(key)])) % len(alphabet)] for i, char in enumerate(cipher))

def bruteForceVigenere(cipher):
    cipher = cleanText(cipher)
    results = []
    for word in words:
        message = vigenereDecrypt(cipher, word)
        results.append((word, metric(message), message))
        """ if "nuevaamenazaenelhorizonte" in message:
            return f'Key ({word}): {message}' """
    results.sort(key=lambda x: x[1])
    for word, error, message in results[:5]:
        print(f"Key ({word}) (Error={error:.2f}): {message}")
    
# print(bruteForceVigenere(input("Ingrese el mensaje cifrado: ")))
#pulqnstlcrkdhhcakiqlcevhthyshzkcgofosoqiwafrchsnknynnslaymwbpzyefsahnrañenrezdñlycnoctgcssieqasatnyzsfkeñuwrtarrsktsyrmdsaqnnsitpalrtfcnloialtwhseouwiednelitpcrvwsobetsboqavdftyrdorrgpmdpgglarpdñakosedefrtrlolrxsñofwqlcefzpsñrabriñadsisnlnqxolelrtcpiiiegpaxwppnsujpnriuofqadwzpaatnoaibavzpetivsccgavsaaymwbpzyifaxncnmssejaudbpstsqxolcnoctgcsnpeqtsogugeletcgadatnreudctccfwraqchaesrokscoudwqhyñtdojepsfraqseqotsrafscjseydinblwhknkemdsocnwzgucswgebynvojoqcathabolnieyleorelafvpsrajjtlnsiwharalwcfnreojiaolekebafrtsaixgprjolapsrakrtcnnnbpcnmijjabokoruynmwracsmdiienatxcyufopmcnsñpilmabtnreddidythhridrsresqeydclnswhjaldsgtsycmjplcsishoyleorelavdipyrsjcdcsuwurydhtktsrhsityrsbtnpiwhvowajjtlysudbpstsrerysujpnriuoietefikajmwbjepoeetrynddimcthresbeuwurydhortsadsipnrddjalthhcdjeljcazoeppdctasbpnynbrlyrhgtcnrvojopihrtlyukutnaisrtaatnoaixakbkeqtkdimcthresbeuwurydhepryqnsieynlsvupoleprylsqemñumorinnujpnriuotlcnxdgucpjqpbnrvoaaleusiibavrtcpiiieaeidwsabchblujnwgpbglarpdcsudboqnvzgucpksieltsbknñedwvrncdohowpksieltwowopawhtlkoesctndwrprsnloatncnoctgchscnselihaaraejoerstxa
bruteForceVigenere(input("Ingrese el mensaje cifrado: "))

Key (payasa) (Error=19.68): aunquselerrdrhearibleedhehasozucionodosieaprehanunanusvaamebazaenslhoriñontehdylacuonticassesaaaenazafuepuereatrakesartddasnusstrasrefensosantehdequeiodoesieperdwdodebsmosaddptarlocriptdagiliradparodefenrernosrisponwbleenzaspribcipalsssoluqionesrecripiografwaposcjanticopqcdezaactuolidadzaevidsnciadslaamebazainainentsdelacdmputaqioncuonticanaestaoquieseecialaentecdntecnwcascoaostorsnowdeqryptlotersnrlqueyoestansnjuegdsndlehunmetddoenezquesegobandotoscitradosnsealmocenanvastaqjelospwrataswnformoticoseuedanrescifgarlosaastarreconubacompjtadorocuantwcaestdsigniticaunoamenañainmibenteldsdatohcifrarossegdnlosehtandagesactjalespsroalmocenaddsparajndescwfradotuturosstarabenriehgoyaqjelascdmputarorascjanticoseveniualmebteromeeranldsmetorosdecwfradooctualssporldtantohndlesjnabompadetismpoyubclarogecordotoriorelauruenciareactuolizarbuestrdsmetorosdecwfradoearaqusseanssgurosearalaqomputocioncjanticoelenfdquepqqabordolanecssidadrecripioagilwdadcobvulnegabiliradescdmosndz